### Module Import and Auto-Reload

In [2]:
# Make sure autoreload is properly set up for auto reload the module if changes
%load_ext autoreload
%autoreload 2

In [3]:
import os, sys
from dotenv import load_dotenv, find_dotenv

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)


_ = load_dotenv(find_dotenv())

# print(os.environ.get('OPENAI_API_KEY'))
print(parent_dir)

/home/ct-admin/Documents/Langgraph/00_AudioProject/02_Call_Center_AI_Agent


In [9]:

from src.Agents.call_center_agent.data_parameter_builder import (
    get_client_data,
    prepare_parameters, 
    analyze_client_behavior,
    ConversationState,
    VerificationStatus
)
data = get_client_data('83906', force_reload=True)

2025-05-29 10:43:50,826 - INFO - Fetching fresh data for user_id: 83906


2025-05-29 10:44:07,912 - INFO - Successfully loaded data for user_id: 83906


In [10]:
data["account_aging"]

{'x0': '0.0000',
 'x30': '4030.0000',
 'x60': '0.0000',
 'x90': '0.0000',
 'x120': '0.0000',
 'xbalance': '4030.0000'}

### Test

In [11]:
# ./example_usage_short.py
"""
Compact test to see full prompts for each call center step.
"""

from src.Agents.call_center_agent.call_scripts import ScriptType, CallStep
from src.Agents.call_center_agent.data_parameter_builder import (
    get_client_data,
    prepare_parameters, 
    ConversationState,
    VerificationStatus
)
from src.Agents.call_center_agent.prompts import get_step_prompt

def test_all_step_prompts():
    """Test all step prompts with full content display."""
    
    # Setup
    user_id = "83906"
    agent_name = "Sarah Mitchell"
    script_type = "ratio_1_inflow"
    
    print("🔄 Loading client data...")
    client_data = get_client_data(user_id)
    client_name = client_data.get('profile', {}).get('client_info', {}).get('client_full_name', 'Client')
    print(f"✅ Loaded data for: {client_name}")
    
    # Initialize state as fully verified
    state = ConversationState()
    state.name_verification_status = VerificationStatus.VERIFIED.value
    state.details_verification_status = VerificationStatus.VERIFIED.value
    state.add_matched_field("id_number")
    state.add_objection("no_money")
    state.update_emotional_state("cooperative")
    
    # All steps to test
    steps = [
        "introduction",
        "name_verification", 
        "details_verification",
        "reason_for_call",
        "negotiation",
        "promise_to_pay",
        "debicheck_setup",
        "payment_portal",
        "subscription_reminder",
        "client_details_update",
        "referrals",
        "further_assistance",
        "query_resolution",
        "escalation",
        "cancellation",
        "closing"
    ]
    
    print(f"\n🧪 Testing {len(steps)} call center steps:")
    print("=" * 80)
    
    for i, step in enumerate(steps, 1):
        print(f"\n📋 STEP {i:2d}: {step.upper()}")
        print("-" * 50)
        
        try:
            # Build parameters
            parameters = prepare_parameters(
                client_data=client_data,
                current_step=step,
                state=state.to_dict(),
                script_type=script_type,
                agent_name=agent_name
            )
            
            # Generate full prompt
            full_prompt = get_step_prompt(step, parameters)
            
            # Display prompt (truncate if too long)
            if len(full_prompt) > 800:
                print(f"🎯 PROMPT: {full_prompt[:400]}...")
                print(f"...{full_prompt[-400:]}")
            else:
                print(f"🎯 PROMPT: {full_prompt}")
            
            # Show key parameters
            key_params = {
                "client_name": parameters.get("client_name", "N/A"),
                "outstanding_amount": parameters.get("outstanding_amount", "N/A"),
                "urgency_level": parameters.get("urgency_level", "N/A"),
                "key_motivators": parameters.get("key_motivators", "N/A")
            }
            print(f"📊 KEY PARAMS: {key_params}")
            
        except Exception as e:
            print(f"❌ ERROR: {e}")
        
        print("-" * 50)
    
    print(f"\n✅ Tested all {len(steps)} steps successfully!")

def test_script_variations_short():
    """Test different script types with key differences highlighted."""
    
    print("\n🎭 SCRIPT VARIATIONS TEST")
    print("=" * 60)
    
    user_id = "83906"
    agent_name = "Sarah"
    
    client_data = get_client_data(user_id)
    state = ConversationState()
    state.name_verification_status = VerificationStatus.VERIFIED.value
    state.details_verification_status = VerificationStatus.VERIFIED.value
    
    script_types = [
        ("ratio_1_inflow", "First missed payment"),
        ("ratio_2_3_inflow", "2-3 months overdue"), 
        ("pre_legal_120_plus", "Pre-legal 120+ days"),
        ("pre_legal_150_plus", "Legal attorney involved")
    ]
    
    for script_type, description in script_types:
        print(f"\n📝 {description.upper()}")
        print("-" * 30)
        
        parameters = prepare_parameters(
            client_data=client_data,
            current_step="reason_for_call",
            state=state.to_dict(),
            script_type=script_type,
            agent_name=agent_name
        )
        
        # Show script content
        script_content = parameters.get("script_content", "No script content")
        print(f"💬 SCRIPT: {script_content}")
        
        # Show tactical differences
        urgency = parameters.get("urgency_level", "None")
        motivators = parameters.get("key_motivators", "None")
        
        print(f"⚡ URGENCY: {urgency}")
        print(f"🎯 MOTIVATORS: {motivators}")

def test_conversation_intelligence():
    """Test conversation intelligence features."""
    
    print("\n🧠 CONVERSATION INTELLIGENCE TEST")
    print("=" * 50)
    
    from src.Agents.call_center_agent.data_parameter_builder import (
        detect_emotional_state,
        detect_objections,
        analyze_payment_conversation
    )
    
    # Simulate conversation messages
    test_messages = [
        {"role": "ai", "content": "Can we debit R500 today?"},
        {"role": "human", "content": "I don't have money right now, I'm really stressed about this"},
        {"role": "ai", "content": "I understand. What amount could you manage?"},
        {"role": "human", "content": "Maybe R200 next week when I get paid"}
    ]
    
    print("💭 Test conversation:")
    for msg in test_messages:
        speaker = "AGENT" if msg["role"] == "ai" else "CLIENT"
        print(f"   {speaker}: {msg['content']}")
    
    # Analyze conversation
    emotional_state = detect_emotional_state(test_messages)
    objections = detect_objections(test_messages)
    payment_analysis = analyze_payment_conversation(test_messages, 500.0)
    
    print(f"\n🎭 EMOTIONAL STATE: {emotional_state}")
    print(f"🚫 OBJECTIONS: {objections}")
    print(f"💰 PAYMENT ANALYSIS: {payment_analysis}")

def test_outstanding_calculation():
    """Test outstanding amount calculation."""
    
    print("\n💰 OUTSTANDING AMOUNT CALCULATION TEST")
    print("=" * 50)
    
    from src.Agents.call_center_agent.data_parameter_builder import (
        calculate_outstanding_amount,
        format_outstanding_amount
    )
    
    # Test data
    test_aging_data = {
        "x0": "100.00",      # Current (not overdue)
        "x30": "200.00",     # 1-30 days overdue
        "x60": "150.00",     # 31-60 days overdue
        "x90": "100.00",     # 61-90 days overdue
        "x120": "50.00",     # 91+ days overdue
        "xbalance": "600.00" # Total balance
    }
    
    outstanding_float = calculate_outstanding_amount(test_aging_data)
    formatted_amount = format_outstanding_amount({"account_aging": test_aging_data})
    
    print(f"📊 AGING BREAKDOWN:")
    print(f"   Current (x0): R {test_aging_data['x0']}")
    print(f"   30 days: R {test_aging_data['x30']}")
    print(f"   60 days: R {test_aging_data['x60']}")
    print(f"   90 days: R {test_aging_data['x90']}")
    print(f"   120+ days: R {test_aging_data['x120']}")
    print(f"   TOTAL: R {test_aging_data['xbalance']}")
    
    print(f"\n💯 CALCULATION:")
    print(f"   Outstanding = Total - Current")
    print(f"   Outstanding = R {test_aging_data['xbalance']} - R {test_aging_data['x0']}")
    print(f"   Outstanding = R {outstanding_float:.2f}")
    print(f"   Formatted: {formatted_amount}")

def quick_test():
    """Quick test of key functionality."""
    
    print("🚀 QUICK CALL CENTER TEST")
    print("=" * 40)
    
    # Test 3 key steps
    key_steps = ["reason_for_call", "negotiation", "promise_to_pay"]
    
    user_id = "83906"
    client_data = get_client_data(user_id)
    state = ConversationState()
    state.name_verification_status = VerificationStatus.VERIFIED.value
    state.details_verification_status = VerificationStatus.VERIFIED.value
    
    for step in key_steps:
        print(f"\n🎯 {step.upper()}")
        
        parameters = prepare_parameters(
            client_data=client_data,
            current_step=step,
            state=state.to_dict(),
            script_type="ratio_1_inflow",
            agent_name="Sarah"
        )
        
        prompt = get_step_prompt(step, parameters)
        
        # Show first 200 chars of prompt
        print(f"📝 PROMPT: {prompt[:200]}...")
        
        # Show key fields
        outstanding = parameters.get("outstanding_amount", "N/A")
        urgency = parameters.get("urgency_level", "N/A")
        print(f"💰 Amount: {outstanding} | ⚡ Urgency: {urgency}")

if __name__ == "__main__":
    print("🎯 CALL CENTER SYSTEM - COMPACT TESTS")
    print("=" * 60)
    
    # Choose which test to run
    test_choice = input("\nChoose test:\n1. All step prompts\n2. Script variations\n3. Conversation intelligence\n4. Outstanding calculation\n5. Quick test\n\nEnter number (1-5): ")
    
    if test_choice == "1":
        test_all_step_prompts()
    elif test_choice == "2":
        test_script_variations_short()
    elif test_choice == "3":
        test_conversation_intelligence()
    elif test_choice == "4":
        test_outstanding_calculation()
    elif test_choice == "5":
        quick_test()
    else:
        print("Running quick test by default...")
        quick_test()
    
    print("\n✅ Test completed!")

🎯 CALL CENTER SYSTEM - COMPACT TESTS


2025-05-29 10:44:41,850 - INFO - Using cached data for user_id: 83906
2025-05-29 10:44:41,852 - INFO - Verification field matched: id_number
2025-05-29 10:44:41,853 - INFO - Objection added: no_money
2025-05-29 10:44:41,855 - INFO - Emotional state updated to: cooperative
2025-05-29 10:44:41,856 - INFO - Outstanding calculation:
2025-05-29 10:44:41,857 - INFO -   Total balance (xbalance): R 4030.00
2025-05-29 10:44:41,859 - INFO -   Current balance (x0): R 0.00
2025-05-29 10:44:41,860 - INFO -   Outstanding amount: R 4030.00
2025-05-29 10:44:41,861 - INFO -   Breakdown - 30d: R4030.00, 60d: R0.00, 90d: R0.00, 120d+: R0.00
2025-05-29 10:44:41,862 - INFO - Outstanding calculation:
2025-05-29 10:44:41,863 - INFO -   Total balance (xbalance): R 4030.00
2025-05-29 10:44:41,863 - INFO -   Current balance (x0): R 0.00
2025-05-29 10:44:41,864 - INFO -   Outstanding amount: R 4030.00
2025-05-29 10:44:41,865 - INFO -   Breakdown - 30d: R4030.00, 60d: R0.00, 90d: R0.00, 120d+: R0.00
2025-05-29 10

🔄 Loading client data...
✅ Loaded data for: Linda Oelofse

🧪 Testing 16 call center steps:

📋 STEP  1: INTRODUCTION
--------------------------------------------------
🎯 PROMPT: 
<role>
You are Sarah Mitchell, a professional debt collection specialist at Cartrack's Accounts Department.
</role>

<context>
- Making OUTBOUND call regarding overdue account
- CRITICAL: Cannot discuss ANY account details until client identity is FULLY verified
- Current step: introduction
</context>

<communication_style>
- Professional, confident, and solution-focused
- MAXIMUM 20 WORDS per re...
...se?"
</script_foundation>

<approach>
"Good day, you are speaking to Sarah Mitchell from Cartrack Accounts Department. May I speak to Mrs Linda Oelofse, please?"
</approach>

<style>
- MAXIMUM 15 words
- Professional greeting
- Clear identification
- Direct request for specific person
</style>

<success_criteria>
Professional introduction completed, requesting specific client.
</success_criteria>

📊 KEY PARAMS: {

In [6]:
user_id = "83906"
agent_name = "Sarah Mitchell"
script_type = "ratio_1_inflow"  # Standard first missed payment
client_data = get_client_data(user_id)


2025-05-27 13:11:42,719 - INFO - Using cached data for user_id: 83906


In [12]:
state = ConversationState()
state.add_objection("no_money")
state.update_emotional_state("defensive")
state.update_payment_willingness("low")
parameters = prepare_parameters(
        client_data=client_data,
        current_step="negotiation",
        state=state.to_dict(),
        script_type=script_type,
        agent_name=agent_name
    )
    
reason_prompt = get_step_prompt("reason_for_call", parameters)
print(reason_prompt)

2025-05-27 13:13:03,915 - INFO - Objection added: no_money
2025-05-27 13:13:03,915 - INFO - Emotional state updated to: defensive
2025-05-27 13:13:03,916 - INFO - Payment willingness updated to: low
2025-05-27 13:13:03,916 - INFO - Built parameters for user_id: 83906, step: negotiation



<role>
You are Sarah Mitchell, a professional debt collection specialist at Cartrack's Accounts Department.
</role>

<context>
- Making OUTBOUND call regarding overdue account
- CRITICAL: Cannot discuss ANY account details until client identity is FULLY verified
- Client may be defensive, evasive, or emotional - remain professional but assertive
- Current step: negotiation
</context>

<communication_style>
- Professional, confident, and solution-focused
- Concise responses (max 30 words unless detailed explanation needed)
- Acknowledge emotions but redirect to resolution
- Use "I understand" to validate, then guide to next step
</communication_style>

<restrictions>
- NO account details or amounts until verification complete
- NO discussing reason for call until identity confirmed
- Stay focused but adapt tone to client response
</restrictions>

<task>
Clearly communicate account status and required payment amount.
</task>

<script_foundation>
Core message: ""
</script_foundation>

<c